In [ ]:
import os
import numpy as np
from scipy.io import loadmat

folder_path = r"..\los_data"  # 你的文件夹路径
file_list = [f for f in os.listdir(folder_path) if f.endswith('.mat')]
file_list = sorted(file_list)

# 选择处理逻辑：
# 1 -> 先差分再转置
# 2 -> 先转置再差分
process_mode = 2  # 改成 2 可对比另一个顺序

data = []

for i, file_name in enumerate(file_list):
    full_path = os.path.join(folder_path, file_name)
    mat_contents = loadmat(full_path)

    if 'data_Ineed' in mat_contents:
        raw_data = mat_contents['data_Ineed']

        if process_mode == 1:
            # 先差分再转置
            diff_data = np.empty_like(raw_data)
            diff_data[0] = raw_data[0]
            diff_data[1:] = raw_data[1:] - raw_data[:-1]
            processed_data = diff_data.T

            # 按 320 个信号重构
            num_signals, num_samples = processed_data.shape
            block_size = 320
            num_blocks = num_signals // block_size
            processed_data = processed_data[:num_blocks * block_size, :]
            processed_data = processed_data.reshape((block_size, num_blocks * num_samples)).T

        elif process_mode == 2:
            # 先转置
            transposed = raw_data.T

            # 按 320 个信号重构
            num_signals, num_samples = transposed.shape
            block_size = 320
            num_blocks = num_signals // block_size
            reshaped_data = transposed[:num_blocks * block_size, :]
            reshaped_data = reshaped_data.reshape((block_size, num_blocks * num_samples)).T

            # 转置，让时间轴在 axis=0（和 process_mode=1 对齐）
            reshaped_data_T = reshaped_data.T

            # 差分
            diff_data = np.empty_like(reshaped_data_T)
            diff_data[0] = reshaped_data_T[0]
            diff_data[1:] = reshaped_data_T[1:] - reshaped_data_T[:-1]

            # 再转置回来
            processed_data = diff_data.T


        else:
            raise ValueError("process_mode 必须是 1 或 2")

        data.append(processed_data)
    else:
        print(f"Warning: {file_name} 里没有 'data_Ineed' 变量！")

# 打印检查
for i, d in enumerate(data):
    print(f"第{i}个文件（{file_list[i]}）的数据 shape (处理后): {d.shape}, 前5个值: {d[:5,0]}")


In [ ]:
import numpy as np

# 假设data是从之前的加载代码中获取的
# data = <从前一部分代码中加载的数据>

SNR_dB = -5  # 设定信噪比（dB）

noisy_data = []  # 用来存储加噪声后的数据

for d in data:
    # 计算信号的标准差
    signal_std = np.std(d)
    
    # 计算噪声的标准差，根据信噪比（SNR）
    noise_std = signal_std / (10 ** (SNR_dB / 20))  # 根据 SNR(dB) 计算噪声标准差
    
    # 生成高斯噪声并加到原信号上
    noise = np.random.normal(0, noise_std, d.shape)  # 均值为0，标准差为噪声标准差
    noisy_signal = d + noise  # 添加噪声
    
    noisy_data.append(noisy_signal)  # 保存加噪声后的信号

# 打印检查加噪声后的数据
print("原始数据和加噪声数据示例：")
print(f"原始信号样本（第一个信号）：{data[0][0][:10]}")  # 打印前10个采样点
print(f"加噪声信号样本（第一个信号）：{noisy_data[0][0][:10]}")  # 打印前10个采样点


In [ ]:
import numpy as np

def compute_doppler_shift(v, fc):
    """
    根据移动速度 v (m/s) 和载波频率 fc (Hz) 计算多普勒频移
    """
    c = 3e8  # 光速 m/s
    return (v / c) * fc

def add_doppler_shift(signal, fd, fs):
    """
    给IQ信号加多普勒频移
    signal: shape (num_channels, num_samples)，复数IQ信号
    fd: 多普勒频移 (Hz)
    fs: 采样率 (Hz)
    """
    num_channels, num_samples = signal.shape
    t = np.arange(num_samples) / fs  # 时间轴
    doppler_phase = np.exp(1j * 2 * np.pi * fd * t)  # 复指数
    return signal * doppler_phase

# 例子
fs = 20e6  # 采样率，比如1 MHz
fc = 2.4e9  # 载波频率，比如2.4 GHz Wi-Fi
v = 120  # 移动速度，比如30 m/s (~108 km/h)

fd = compute_doppler_shift(v, fc)  # 先计算多普勒频移
print(f"计算得到的多普勒频移 fd = {fd:.2f} Hz")

# 假设 data 是你之前读出来的list，每个是 shape (信号数量, 采样点数)
data_with_doppler = []
for sig in noisy_data:
    shifted_sig = add_doppler_shift(sig, fd, fs)
    data_with_doppler.append(shifted_sig)

print("添加多普勒频移后的信号：")
print(data_with_doppler)


In [ ]:
import torch
import numpy as np
from torch.utils.data import DataLoader, TensorDataset, random_split

# 假设 data 已经是你的读取好的列表，每个元素是 (总信号数量, 320)

all_data = []
all_labels = []

for device_idx, device_signals in enumerate(data_with_doppler):
    # device_signals: (总信号数量, 320)，比如 (7014, 320)
    device_labels = np.full((device_signals.shape[0],), device_idx)  # 每条信号的标签都是设备编号
    
    all_data.append(torch.tensor(device_signals, dtype=torch.float32))
    all_labels.append(torch.tensor(device_labels, dtype=torch.long))

# 把20个设备的数据和标签拼接在一起
all_data = torch.cat(all_data, dim=0)  # (总样本数, 320)
all_labels = torch.cat(all_labels, dim=0)  # (总样本数, )

print(f"最终数据 shape: {all_data.shape}")
print(f"最终标签 shape: {all_labels.shape}")

# 构建 PyTorch Dataset
dataset = TensorDataset(all_data, all_labels)

# 划分训练集和验证集，假设验证集占20%
train_size = int(0.8 * len(dataset))  # 80% 训练集
val_size = len(dataset) - train_size  # 剩下的 20% 验证集

# 使用 random_split 划分数据集
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# 打印一下数据集的基本信息
print(f"训练集大小: {len(train_dataset)}")
print(f"验证集大小: {len(val_dataset)}")


In [ ]:
# ==== 数据增强：对训练集添加高斯噪声 ====

def add_noise_to_dataset(dataset, noise_std=0.01):
    noisy_data = []
    noisy_labels = []
    for x, y in dataset:
        noise = torch.randn_like(x) * noise_std
        noisy_x = x + noise
        noisy_data.append(noisy_x)
        noisy_labels.append(y)
    noisy_data = torch.stack(noisy_data)
    noisy_labels = torch.tensor(noisy_labels)
    return TensorDataset(noisy_data, noisy_labels)

train_dataset = add_noise_to_dataset(train_dataset, noise_std=0.01)  # 可调节 std

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns
from torch.utils.data import DataLoader, TensorDataset, Subset
from datetime import datetime
from tqdm import tqdm
from sklearn.model_selection import KFold

# 假设 SNR_dB 和 fd 已经在之前定义
SNR_dB = globals().get('SNR_dB', 'no')
fd = globals().get('fd', 'no')

# 初始化模型参数
input_dim = 320  # 每个信号有320个采样点
num_heads = 4
num_layers = 2
num_classes = 20
dropout = 0.4

# 训练参数
batch_size = 128
num_epochs = 100
learning_rate = 1e-4
patience = 5

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
script_name = "cross_DCTF"
folder_name = f"{timestamp}_{script_name}_SNR{SNR_dB}dB_fd{fd}_classes_{num_classes}_Transformer"
save_folder = os.path.join(os.getcwd(), "training_results", folder_name)
os.makedirs(save_folder, exist_ok=True)
results_file = os.path.join(save_folder, "results.txt")

with open(results_file, "w") as f:
    f.write(f"=== Experiment Summary ===\n")
    f.write(f"Feature Folder: trajectory_plots\n")
    f.write(f"Timestamp: {timestamp}\n")
    f.write(f"Total Classes: {num_classes}\n")
    f.write(f"SNR: {SNR_dB} dB\n")
    f.write(f"fd (Doppler shift): {fd} Hz\n")

# 检查 GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 定义 Transformer 模型
class SignalTransformer(nn.Module):
    def __init__(self, input_dim, num_heads, num_layers, num_classes, dropout=0.1):
        super(SignalTransformer, self).__init__()
        self.embedding = nn.Linear(input_dim, input_dim)
        self.transformer = nn.Transformer(
            d_model=input_dim,
            nhead=num_heads,
            num_encoder_layers=num_layers,
            dropout=dropout,
            batch_first=True
        )
        self.fc = nn.Linear(input_dim, num_classes)
    
    def forward(self, x):
        x = self.embedding(x)
        x = self.transformer(x, x)
        if len(x.shape) == 3:
            x = x[:, -1, :]
        x = self.fc(x)
        return x

# K折交叉验证
n_splits = 5
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)
fold_results = []

# 假设 train_dataset 和 val_dataset 已经定义
for fold, (train_idx, val_idx) in enumerate(kfold.split(train_dataset)):
    print(f"\n====== Fold {fold+1}/{n_splits} ======")
    
    train_subset = Subset(train_dataset, train_idx)
    val_subset = Subset(train_dataset, val_idx)
    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, drop_last=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=True)
    
    model = SignalTransformer(input_dim, num_heads, num_layers, num_classes, dropout).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
    
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []
    best_val_loss = float('inf')
    patience_counter = 0
    
    for epoch in range(num_epochs):
        model.train()
        running_train_loss, correct_train, total_train = 0.0, 0, 0
        
        with tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch") as tepoch:
            for inputs, labels in tepoch:
                inputs = inputs.to(device, dtype=torch.float)
                labels = labels.to(device, dtype=torch.long)
                
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                
                running_train_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total_train += labels.size(0)
                correct_train += (predicted == labels).sum().item()
                train_accuracy = 100 * correct_train / total_train
                tepoch.set_postfix(train_loss=running_train_loss / (tepoch.n+1), train_accuracy=train_accuracy)
        
        epoch_train_loss = running_train_loss / len(train_loader)
        epoch_train_acc = 100 * correct_train / total_train
        train_losses.append(epoch_train_loss)
        train_accuracies.append(epoch_train_acc)
        
        # 验证
        model.eval()
        running_val_loss, correct_val, total_val = 0.0, 0, 0
        with torch.no_grad():
            with tqdm(val_loader, desc="Validation", unit="batch") as vepoch:
                for val_inputs, val_labels in vepoch:
                    val_inputs = val_inputs.to(device, dtype=torch.float)
                    val_labels = val_labels.to(device, dtype=torch.long)
                    val_outputs = model(val_inputs)
                    val_loss = criterion(val_outputs, val_labels)
                    running_val_loss += val_loss.item()
                    _, val_predicted = torch.max(val_outputs, 1)
                    total_val += val_labels.size(0)
                    correct_val += (val_predicted == val_labels).sum().item()
                    val_accuracy = 100 * correct_val / total_val
                    vepoch.set_postfix(val_loss=running_val_loss / (len(val_loader)+1), val_accuracy=val_accuracy)
        
        epoch_val_loss = running_val_loss / len(val_loader)
        epoch_val_acc = 100 * correct_val / total_val
        val_losses.append(epoch_val_loss)
        val_accuracies.append(epoch_val_acc)
        
        print(f"Epoch {epoch+1}/{num_epochs}: Train Loss={epoch_train_loss:.4f}, Train Acc={epoch_train_acc:.2f}%, Val Loss={epoch_val_loss:.4f}, Val Acc={epoch_val_acc:.2f}%")
        
        with open(results_file, "a") as f:
            f.write(f"Epoch {epoch+1} | Train Loss: {epoch_train_loss:.4f} | Train Acc: {epoch_train_acc:.2f}% | Val Loss: {epoch_val_loss:.4f} | Val Acc: {epoch_val_acc:.2f}%\n")
        
        # Early stopping
        if epoch_val_loss < best_val_loss:
            best_val_loss = epoch_val_loss
            patience_counter = 0
        else:
            patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}.")
            break
        
        scheduler.step()
    
    fold_results.append(max(val_accuracies))
    
    # 绘制损失和准确度曲线
    plt.figure()
    plt.plot(range(1, len(train_losses)+1), train_losses, label='Train Loss')
    plt.plot(range(1, len(val_losses)+1), val_losses, label='Val Loss')
    plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.legend()
    plt.savefig(os.path.join(save_folder, f"fold_{fold+1}_loss_curve.png")); plt.close()
    
    plt.figure()
    plt.plot(range(1, len(train_accuracies)+1), train_accuracies, label='Train Acc')
    plt.plot(range(1, len(val_accuracies)+1), val_accuracies, label='Val Acc')
    plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.legend()
    plt.savefig(os.path.join(save_folder, f"fold_{fold+1}_accuracy_curve.png")); plt.close()
    
    # 混淆矩阵
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for val_inputs, val_labels in val_loader:
            val_inputs = val_inputs.to(device, dtype=torch.float)
            val_labels = val_labels.to(device, dtype=torch.long)
            val_outputs = model(val_inputs)
            _, val_predicted = torch.max(val_outputs, 1)
            all_preds.extend(val_predicted.cpu().numpy())
            all_labels.extend(val_labels.cpu().numpy())
    
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(10,8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Fold {fold+1} Confusion Matrix')
    plt.xlabel('Predicted'); plt.ylabel('True')
    plt.savefig(os.path.join(save_folder, f"fold_{fold+1}_confusion_matrix.png")); plt.close()
    np.savetxt(os.path.join(save_folder, f"fold_{fold+1}_confusion_matrix.txt"), cm, fmt='%d')

# 汇总结果
avg_acc = np.mean(fold_results)
with open(results_file, "a") as f:
    f.write("\n=== Cross-Validation Results ===\n")
    for i, acc in enumerate(fold_results):
        f.write(f"Fold {i+1}: Best Val Accuracy: {acc:.2f}%\n")
    f.write(f"\nAverage Best Val Accuracy: {avg_acc:.2f}%\n")

print(f"\n=== Cross-Validation Summary ===")
for i, acc in enumerate(fold_results):
    print(f"Fold {i+1}: Best Val Accuracy = {acc:.2f}%")
print(f"Average Best Val Accuracy = {avg_acc:.2f}%")
